In [1]:
import tensorflow as tf
import os
import numpy as np
import random
from tensorflow.keras.applications import resnet
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

from pyexcel.cookbook import merge_all_to_a_book
# import pyexcel.ext.xlsx # no longer required if you use pyexcel >= 0.2.2 
import glob

def get_embedding_module(imageSize):
    # construct the input layer and pass the inputs through a
    # pre-processing layer
    inputs = keras.Input(imageSize + (3,))
    x = resnet.preprocess_input(inputs)

    # fetch the pre-trained resnet 50 model and freeze the weights
    baseCnn = resnet.ResNet50(weights="imagenet", include_top=False)
    baseCnn.trainable=False

    # pass the pre-processed inputs through the base cnn and get the
    # extracted features from the inputs
    extractedFeatures = baseCnn(x)
    # pass the extracted features through a number of trainable layers
    x = layers.GlobalAveragePooling2D()(extractedFeatures)
    x = layers.Dense(units=1024, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(units=256, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(units=128)(x)
    # build the embedding model and return it
    embedding = keras.Model(inputs, outputs, name="embedding")
    return embedding

def get_siamese_network(imageSize, embeddingModel):
    # build the anchor, positive and negative input layer
    anchorInput = keras.Input(name="anchor", shape=imageSize + (3,))
    positiveInput = keras.Input(name="positive", shape=imageSize + (3,))
    negativeInput = keras.Input(name="negative", shape=imageSize + (3,))
    # embed the anchor, positive and negative images
    anchorEmbedding = embeddingModel(anchorInput)
    positiveEmbedding = embeddingModel(positiveInput)
    negativeEmbedding = embeddingModel(negativeInput)
    # build the siamese network and return it
    siamese_network = keras.Model(
        inputs=[anchorInput, positiveInput, negativeInput],
        outputs=[anchorEmbedding, positiveEmbedding, negativeEmbedding]
    )
    return siamese_network

class SiameseModel(keras.Model):
    def __init__(self, siameseNetwork, margin, lossTracker):
        super().__init__()
        self.siameseNetwork = siameseNetwork
        self.margin = margin
        self.lossTracker = lossTracker
    def _compute_distance(self, inputs):
        (anchor, positive, negative) = inputs
        # embed the images using the siamese network
        embeddings = self.siameseNetwork((anchor, positive, negative))
        anchorEmbedding = embeddings[0]
        positiveEmbedding = embeddings[1]
        negativeEmbedding = embeddings[2]
        # calculate the anchor to positive and negative distance
        apDistance = tf.reduce_sum(
            tf.square(anchorEmbedding - positiveEmbedding), axis=-1
        )
        anDistance = tf.reduce_sum(
            tf.square(anchorEmbedding - negativeEmbedding), axis=-1
        )

        # return the distances
        return (apDistance, anDistance)
    def _compute_loss(self, apDistance, anDistance):
        loss = apDistance - anDistance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss
    def call(self, inputs):
        # compute the distance between the anchor and positive,
        # negative images
        (apDistance, anDistance) = self._compute_distance(inputs)
        return (apDistance, anDistance)
    def train_step(self, inputs):
        with tf.GradientTape() as tape:
            # compute the distance between the anchor and positive,
            # negative images
            (apDistance, anDistance) = self._compute_distance(inputs)
            # calculate the loss of the siamese network
            loss = self._compute_loss(apDistance, anDistance)
        # compute the gradients and optimize the model
        gradients = tape.gradient(
            loss,
            self.siameseNetwork.trainable_variables)
        self.optimizer.apply_gradients(
            zip(gradients, self.siameseNetwork.trainable_variables)
        )
        # update the metrics and return the loss
        self.lossTracker.update_state(loss)
        return {"loss": self.lossTracker.result()}
    def test_step(self, inputs):
        # compute the distance between the anchor and positive,
        # negative images
        (apDistance, anDistance) = self._compute_distance(inputs)
        # calculate the loss of the siamese network
        loss = self._compute_loss(apDistance, anDistance)

        # update the metrics and return the loss
        self.lossTracker.update_state(loss)
        return {"loss": self.lossTracker.result()}
    @property
    def metrics(self):
        return [self.lossTracker]

In [3]:
import cv2
import numpy as np
import os
from pathlib import Path
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from time import sleep
from imutils.video import VideoStream
import imutils

IMAGES_PATH = 'ImageAttendance'
MODEL_PATH = 'siamese_network_collected_data_64'
CASCADE_PATH = 'haarcascade_frontalface_default.xml'
THRESHOLD = 0.05

# Show waiting screen
waiting = np.zeros(500, np.uint8)
cv2.imshow("WebCam", waiting)

# Load the model
modelPath = MODEL_PATH
print(f"[INFO] loading the siamese network from {modelPath}...")
siameseNetwork = keras.models.load_model(filepath=modelPath)
siameseModel = SiameseModel(
	siameseNetwork=siameseNetwork,
	margin=0.5,
	lossTracker=keras.metrics.Mean(name="loss"),
)

# Load Har Cascade face detector
detector = cv2.CascadeClassifier(CASCADE_PATH)

# Find Face Position
def find_faces(img):
    """
    A Function to find faces position in @img

    Args:
        - img: an image
    """
    gray_faces = []
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # detect faces in the grayscale frame
    rects = detector.detectMultiScale(gray, scaleFactor=1.1, 
        minNeighbors=5, minSize=(30, 30))
    # Reordering the image
    for (x, y, w, h) in rects:
        face = gray[y:y + h, x:x + w]
        gray_face = np.zeros((*face.shape[:2],3), np.uint8)
        # print(gray_face.shape, face.shape)
        gray_face[:,:,0] = gray_face[:,:,1] = gray_face[:,:,2] = face[:,:]
        
        gray_face = cv2.resize(gray_face, (224,224))
        gray_faces.append(gray_face)
    return gray_faces, rects

def find_embeddings(images):
    """
    A function to find each face embeddings for the faces in each image
    in @images

    Args:
        - images: a list of images
    """
    images_embeddings = []
    for image in images:
        gray_faces, rectsss = find_faces(image)
        for i, gray_face in enumerate(gray_faces):
            cv2.imwrite(f'temp/{i}.jpg', gray_face)

            gray_face = tf.io.read_file(f'temp/{i}.jpg')
            gray_face = tf.image.decode_image(gray_face, channels=3)
            gray_face = tf.image.resize(gray_face, (224,224))
            gray_face = tf.cast(gray_face, tf.float32) / 255.0
            gray_face = tf.expand_dims(gray_face, axis=0)

            embeddings = siameseModel.siameseNetwork((gray_face, gray_face, gray_face))[0]
            images_embeddings.append(embeddings)
    return images_embeddings, rectsss

def markAttendance(name):
    """
    A function to marke attendance of the student @name

    Args:
        - name: the name of the student
    """
    now = datetime.now()
    fileName = "attendance" + str(now.date()) + ".csv"
    with open (fileName ,'a+') as f:
        if os.stat(fileName).st_size == 0:
            f.write("name, date")
        f.seek(0)
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(', ')
            nameList.append(entry[0])
        if name not in nameList:
            dtstring = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name}, {dtstring}') 
    filexlsm = "attendance" + str(now.date()) + ".xlsm"      
    merge_all_to_a_book(glob.glob(fileName), filexlsm)
      

#######################################
# A list to save embeddings
Path = IMAGES_PATH
images = []
ClassNames = []
MyList = os.listdir(Path)
print(MyList)
for cl in MyList:
    curImg = cv2.imread(f'{Path}/{cl}')
    images.append(curImg)
    ClassNames.append(os.path.splitext(cl)[0])
print(ClassNames)
students_embedding, _ =  find_embeddings(images)
arrivals_embedding = []
matched_index = -1
# print("Length of students embeddings: ", len(students_embedding))

print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()
sleep(2)

while True:
    img = vs.read()
    img = imutils.resize(img, width=500)
    matchName = []
    matchDist = []
    matchIndex = []
    arrivals_embedding, rect = find_embeddings([img])

    for j, arrival_embedding in enumerate(arrivals_embedding):
        for i, student_embedding in enumerate(students_embedding):
            distance = tf.reduce_sum(tf.square(student_embedding - arrival_embedding), axis=-1)
            distance = float(distance[0])
            print(f"The disctance with {ClassNames[i]} is:", distance)
            if distance < 0.20:
                matchName.append(ClassNames[i].upper())
                matchDist.append(distance)
                matchIndex.append(rect[j])
        if len(matchDist) != 0:    
                Dist = min(matchDist)
                # print (Dist)
                index = matchDist.index(Dist)
                (x, y, w, h) = rect[j]
                name = matchName[index]
                y1, x2, y2, x1 = y, (y + h) + 50, x + 170, (x + w) - 200
                cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
                cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
                cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
                markAttendance(name)
                print (name)

    cv2.imshow("WebCam",img)
    k = cv2.waitKey(100)
    if k == ord("q"):
        break

vs.stream.release()
cv2.destroyAllWindows()

[INFO] loading the siamese network from siamese_network_collected_data_64...
['had.jpg', 'hosny.png', 'kimo.jpg', 'salah.jpg', 'ziad.jpg']
['had', 'hosny', 'kimo', 'salah', 'ziad']
[INFO] starting video stream...
The disctance with had is: 1.9360756874084473
The disctance with hosny is: 3.37654972076416
The disctance with kimo is: 2.24184513092041
The disctance with salah is: 0.5929044485092163
The disctance with ziad is: 3.4123098850250244
The disctance with had is: 0.418770432472229
The disctance with hosny is: 0.741789698600769
The disctance with kimo is: 0.5078322887420654
The disctance with salah is: 0.16378182172775269
The disctance with ziad is: 0.9137012958526611
SALAH
The disctance with had is: 0.9033197164535522
The disctance with hosny is: 1.3682503700256348
The disctance with kimo is: 0.7854717969894409
The disctance with salah is: 0.11932060122489929
The disctance with ziad is: 1.1738836765289307
SALAH
The disctance with had is: 1.296838402748108
The disctance with hosny i